In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
from datetime import datetime
import time
import os
import re
import sys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

In [2]:
service = Service('/usr/lib/chromium-browser/chromedriver')
options = Options()
options.add_argument('--incognito')
options.add_argument('start-maximized')
driver = webdriver.Chrome(service=service, options=options)
URL=f"https://w.ynet.co.il/news/attackingaza?externalurl=true"
driver.get(URL)
time.sleep(5)

In [3]:
html="a"
new=""
i=0
while new != html:
    i=i+1
    print(f"scroll:{i}")
    new=html
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    html = driver.page_source
    print(f"html length:{len(html)}")
soup = bs(html, features='html.parser')

scroll:1
html length:508298
scroll:2
html length:720238
scroll:3
html length:934654
scroll:4
html length:1147840
scroll:5
html length:1361818
scroll:6
html length:1575742
scroll:7
html length:1789279
scroll:8
html length:2002668
scroll:9
html length:2217923
scroll:10
html length:2432491
scroll:11
html length:2647947
scroll:12
html length:2863977
scroll:13
html length:3002978
scroll:14
html length:3002978


In [4]:
df = pd.DataFrame(columns=['rank','first_name','last_name','age','psik','place','text'])
frames = soup.findAll('a', class_='cardgazaattack-name w-inline-block')
i=0
for  frame in frames:
    i=i+1
    #print (f"row:{i}")
    a=frame.find('div',class_='fallen-name-wrap')
    if(a):
        nameList=a.findAll('div', class_='gazaattack-name')
        v_name_1=nameList[0].text
        v_name_2=nameList[1].text
        v_name_3=nameList[2].text
    else:
        v_name_1=' '
        v_name_2=' '
        v_name_3=' '
    b=frame.find('div',class_='fallen-info-wrap')
    #print(f"{b}")
    if(b):
        agePlaceList=b.findAll('div', class_='gazaattack-place-age')
        v_agePlace_1=agePlaceList[4].text
        v_agePlace_2=agePlaceList[5].text
        v_agePlace_3=agePlaceList[6].text
    else:
        v_agePlace_1=' '
        v_agePlace_2=' '
        v_agePlace_3=' '

    if(frame.find('div',class_='gazaattack-name-story')):
        v_text=frame.find('div',class_='gazaattack-name-story').text.strip()
    else:
        v_text=' '

    #print (f"text:--{v_text}--")
    #print (f"name:--{v_name}--")
    #print (f"place:--{v_place}--")
    df.loc[len(df)] = [v_name_1,v_name_2,v_name_3,v_agePlace_1,v_agePlace_2,v_agePlace_3,v_text]

print (f"number of victims: {i}")
#print (f"{frame}")


number of victims: 954


In [5]:
a=frame.find('div',class_='fallen-name-wrap')
if(a):
    nameList=a.findAll('div', class_='gazaattack-name')
    v_name_1=nameList[0].text
    v_name_2=nameList[1].text
    v_name_3=nameList[2].text
    #v_name=frame.find('div',class_='fallen-name-wrap').text
else:
    v_name=' '
print(f"1:{v_name_1}, 2:{v_name_2}, 3:{v_name_3},")

1:, 2:אופיר, 3:ליבשטיין,


In [6]:
master_df=df.copy()
master_df


,rank,first_name,last_name,age,psik,place,text
0,,יהודית,יצחקי,,",",,
1,,חיים,בנעים,,",",נתיב העשרה,נרצח סמוך לביתו
2,"רס""מ",בוריס,דנילוב,,",",,
3,,משה,בן פורת,,",",,
4,,אברהם אלברט,מילס,,",",,
...,...,...,...,...,...,...,...
949,,מאלק אברהים,אלקרעאן,14,",","אלבאת, פזורה בדואית",נרצח מירי לפזורה הבדואית בנגב
950,,פאטמה,אלטלאקת,35,",",ערערה בנגב,נרצחה באופקים
951,,אביעד,הלוי,29,",",גן דרום,נרצח במסיבה ברעים
952,סגן טפסר,שלום,צבאן,60,",",שדרות,מפקד תחנת כיבוי קריית גת


In [7]:
df.iloc[2,]

rank            רס"מ
first_name     בוריס
last_name     דנילוב
age                 
psik              , 
place               
text                
Name: 2, dtype: object

In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")
master_df.to_csv(f"/home/gilad/Dropbox/SoI/ynet_victims_{timestr}.csv")
driver.close()